导入所需包，准备数据

In [1]:
import pandas as pd
import numpy as np
import pickle
import model_function
import preprocessing
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
with open('./data/train_data.pkl', 'rb') as f:
    train_data_raw = pickle.load(f)
with open('./data/test_data.pkl', 'rb') as f:
    test_data_raw = pickle.load(f)

In [3]:
split_date = round(train_data_raw['date'].unique().shape[0] * 0.9)
valid_data_raw = train_data_raw.loc[train_data_raw['date'] >= split_date]
train_data_raw = train_data_raw.loc[train_data_raw['date'] < split_date]

In [4]:
X_train_raw = train_data_raw.drop(columns=['y']).reset_index(drop=True)
y_train = train_data_raw[['code', 'date', 'y']].reset_index(drop=True)
X_valid_raw = valid_data_raw.drop(columns=['y']).reset_index(drop=True)
y_valid = valid_data_raw[['code', 'date', 'y']].reset_index(drop=True)
X_test_raw = test_data_raw.reset_index(drop=True)

对f_6使用target_encoding编码

In [5]:
start_time = datetime.now()
print("开始时间为:", start_time)

X_train_encoded, target_encoder = preprocessing.target_encode(X_train_raw, 'f_6', y_train['y'])
X_valid_encoded = preprocessing.target_encode(X_valid_raw, 'f_6', encoder=target_encoder)
X_test_encoded = preprocessing.target_encode(X_test_raw, 'f_6', encoder=target_encoder)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-24 17:48:15.382659
结束时间为: 2024-04-24 17:48:18.968245
处理时间为： 0:00:03.585586


对训练特征集做描述性统计

In [6]:
perct = [0.005,0.01,0.02,0.03, 0.04, 0.05, 0.1,0.15,0.25]
perct += [1-a for a in perct]
perct += [0.5]
X_train_encoded.describe(percentiles = perct)

,date,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18
count,4.071914e+06,4.071914e+06,3.660015e+06,4.042132e+06,4.071914e+06,4.071914e+06,4.071914e+06,4.071914e+06,4.066998e+06,4.071601e+06,4.071601e+06,4.070653e+06,4.070657e+06,4.054628e+06,4.071914e+06,4.070834e+06,3.610427e+06,4.071914e+06,3.581510e+06,4.071914e+06
mean,8.299569e+02,1.020225e+00,1.469144e+02,3.060250e+04,4.276906e+00,9.800039e-01,1.000806e+00,-1.495404e-02,6.857498e+00,4.706855e-02,5.508865e-02,1.005105e+00,1.003287e+00,1.595803e+00,1.816358e+08,1.155865e+01,2.370389e-02,9.986466e-01,1.188183e-02,1.000540e+00
std,4.492696e+02,2.576125e-02,3.095289e+03,5.126441e+06,1.005392e+00,2.416416e-02,3.210823e-02,1.690996e-02,1.253369e+02,5.903206e-01,3.068755e+00,3.701825e+00,1.729890e+00,8.583799e+01,4.409331e+08,5.760775e+02,1.181143e+00,1.575991e-02,4.260839e+00,2.412891e-02
min,0.000000e+00,8.335404e-01,1.225800e+00,-7.011802e+02,1.835776e+00,7.105590e-01,7.105590e-01,-7.084265e-02,3.881000e-01,-5.538682e+00,-6.055728e+02,-8.954691e+02,-5.474717e+00,-6.956416e+03,7.075000e+03,1.850000e-02,-1.757517e+01,7.788820e-01,-5.739492e+00,7.776594e-01
0.5%,8.000000e+00,9.691748e-01,5.698807e+00,-8.466107e-01,2.433594e+00,8.998853e-01,8.999781e-01,-7.009279e-02,7.399000e-01,-1.893255e-01,-7.409930e-01,8.460711e-01,9.471765e-01,-3.336842e+01,3.892780e+06,1.705000e-01,-2.469069e+00,9.303887e-01,-2.031957e+00,9.201136e-01
1%,1.700000e+01,9.798793e-01,6.691100e+00,-7.322914e-01,2.555626e+00,9.000000e-01,9.001835e-01,-6.795425e-02,8.169000e-01,-1.223440e-01,-4.012339e-01,9.384646e-01,9.751634e-01,-1.568607e+01,4.992371e+06,2.331000e-01,-2.381345e+00,9.534247e-01,-1.909802e+00,9.318534e-01
2%,3.500000e+01,9.876216e-01,8.234000e+00,-5.891068e-01,2.690701e+00,9.003623e-01,9.168761e-01,-3.693693e-02,9.209000e-01,-7.573300e-02,-2.174960e-01,9.782609e-01,9.905513e-01,-6.435936e+00,6.562512e+06,3.201000e-01,-2.128362e+00,9.676041e-01,-1.744538e+00,9.460505e-01
3%,5.200000e+01,9.913092e-01,9.550200e+00,-4.992984e-01,2.786553e+00,9.089711e-01,9.342875e-01,-3.662287e-02,1.000900e+00,-5.359715e-02,-1.382802e-01,9.896036e-01,9.958710e-01,-3.819850e+00,7.852149e+06,3.963000e-01,-1.893937e+00,9.738966e-01,-1.636496e+00,9.547905e-01
4%,7.000000e+01,9.936167e-01,1.065640e+01,-4.358318e-01,2.864518e+00,9.207833e-01,9.443499e-01,-3.605060e-02,1.067900e+00,-3.860668e-02,-9.078110e-02,9.946416e-01,9.984885e-01,-2.693682e+00,9.023634e+06,4.741000e-01,-1.685659e+00,9.778227e-01,-1.547749e+00,9.606372e-01
5%,8.700000e+01,9.952194e-01,1.159350e+01,-3.847478e-01,2.930295e+00,9.298246e-01,9.511284e-01,-3.542451e-02,1.130700e+00,-2.825400e-02,-6.462400e-02,9.973936e-01,9.999592e-01,-2.009076e+00,1.012879e+07,5.383000e-01,-1.544230e+00,9.804496e-01,-1.477782e+00,9.648291e-01


首先在整个数据集用1%，99%缩尾，接下来用robust_zscore做标准化，最后在界面内部用KNN填充缺失值。

In [7]:
start_time = datetime.now()
print("开始时间为:", start_time)

X_train_win, lower_bound, upper_bound = preprocessing.winsorize_X(X_train_encoded)
X_valid_win = preprocessing.winsorize_X(X_valid_encoded, lower_bound, upper_bound)
X_test_win = preprocessing.winsorize_X(X_test_encoded, lower_bound, upper_bound)

X_train_rzscore, MEDIANS, MADS = preprocessing.robust_zscore(X_train_win)
X_valid_rzscore = preprocessing.robust_zscore(X_valid_win, MEDIANS, MADS)
X_test_rzscore = preprocessing.robust_zscore(X_test_win, MEDIANS, MADS)

X_train_filled = preprocessing.imputer_KNN(X_train_rzscore.drop(columns=['f_10', 'f_11']))
X_valid_filled = preprocessing.imputer_KNN(X_valid_rzscore.drop(columns=['f_10', 'f_11']))
X_test_filled = preprocessing.imputer_KNN(X_test_rzscore.drop(columns=['f_10', 'f_11']))

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-24 17:48:22.276462
结束时间为: 2024-04-24 17:59:49.187512
处理时间为： 0:11:26.911050


选择互信息较高的特征

In [11]:
start_time = datetime.now()
print("开始时间为:", start_time)

X_train_selected, mi_selector = preprocessing.mutual_info_selection(X_train_filled, y_train['y'])
X_valid_selected = preprocessing.mutual_info_selection(X_valid_filled, selector=mi_selector)
X_test_selected = preprocessing.mutual_info_selection(X_test_filled, selector=mi_selector)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-24 18:00:51.879084
结束时间为: 2024-04-24 18:29:21.122165
处理时间为： 0:28:29.243081


对y做截面zscore

In [8]:
start_time = datetime.now()
print("开始时间为:", start_time)

y_train_zscore = preprocessing.zscore_standardization(y_train)
y_valid_zscore = preprocessing.zscore_standardization(y_valid)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-24 17:59:49.203514
结束时间为: 2024-04-24 17:59:53.756935
处理时间为： 0:00:04.553421


构建GRU样本，使用同一支股票连续10天的被选出的10个特征作为模型输入，预测最后一天的y值，保存

In [9]:
def construct_gru_sample(raw_sample_df: pd.DataFrame, window_size: int=10):   
    grouped = raw_sample_df.groupby('code')
    gru_sample_list = []
    for name, group in grouped:
        sorted_group = group.sort_values(by='date')
        rolling_window = sorted_group.set_index('date').rolling(window=window_size)
        for window_data in rolling_window:
            if len(window_data) == window_size:
                date_diff = (window_data.index[-1] - window_data.index[0])
                if date_diff == window_size - 1:
                    gru_sample_list.append(np.array(window_data.reset_index()))
    return np.array(gru_sample_list)

In [12]:
start_time = datetime.now()
print("开始时间为:", start_time)

train_samples = construct_gru_sample(pd.concat([X_train_selected, y_train_zscore['y']], axis=1))
valid_samples = construct_gru_sample(pd.concat([X_valid_selected, y_valid_zscore['y']], axis=1))

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-24 18:29:21.168773
结束时间为: 2024-04-24 18:50:18.641810
处理时间为： 0:20:57.473037


In [13]:
with open('./data/GRU_百分比_RZ_KNN训练样本.pkl', 'wb') as f:
    pickle.dump(train_samples, f)
with open('./data/GRU_百分比_RZ_KNN验证样本.pkl', 'wb') as f:
    pickle.dump(valid_samples, f)

In [14]:
del train_samples
del valid_samples

In [15]:
start_time = datetime.now()
print("开始时间为:", start_time)

test_samples = construct_gru_sample(X_test_selected)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-24 18:54:17.293122
结束时间为: 2024-04-24 19:11:40.406351
处理时间为： 0:17:23.113229


In [16]:
with open('./data/GRU_百分比_RZ_KNN测试样本.pkl', 'wb') as f:
    pickle.dump(test_samples, f)

In [17]:
del test_samples

读取数据，准备训练（给数据用预处理方法改了名）

In [2]:
with open('./data/GRU_百分比_RZ_KNN训练样本.pkl', 'rb') as f:
    train_samples = pickle.load(f)
with open('./data/GRU_百分比_RZ_KNN验证样本.pkl', 'rb') as f:
    valid_samples = pickle.load(f)

In [3]:
# 为节省内存，把后面验证需要的数据先生成，以便早点删除占内存极大的样本变量
GRU_valid_df_ori = pd.DataFrame(valid_samples[:, -1, [0, 1, -1]], columns=['date', 'code', 'y'])

In [4]:
train_targets = train_samples[:, -1, -1].astype(float)
train_features = train_samples[:, :, 2:-1].astype(float)
valid_targets = valid_samples[:, -1, -1].astype(float)
valid_features = valid_samples[:, :, 2:-1].astype(float)

In [5]:
del train_samples
del valid_samples

In [6]:
with open('./data/GRU_百分比_RZ_KNN测试样本.pkl', 'rb') as f:
    test_samples = pickle.load(f)

In [7]:
# 为节省内存，把后面验证需要的数据先生成，以便早点删除占内存极大的样本变量
GRU_test_result_ori = pd.DataFrame(test_samples[:, -1, [0, 1]], columns=['date', 'code'])

In [8]:
test_features = test_samples[:, :, 2:].astype(float)

In [9]:
del test_samples

加载数据

In [11]:
# 定义数据集类
class StockDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float)
        self.targets = torch.tensor(targets, dtype=torch.float)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [12]:
# 准备数据集
train_dataset = StockDataset(train_features, train_targets)
valid_dataset = StockDataset(valid_features, valid_targets)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用的设备:", device)

使用的设备: cuda


In [14]:
# 定义GRU模型
class GRUNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(GRUNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, dropout=0.1, batch_first=True) #加入dropout减少过拟合
        self.bn = nn.BatchNorm1d(hidden_size)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # 默认使用全零张量作为初始隐藏状态
        # 前向传播 GRU
        out, _ = self.gru(x, h0)
        # 取最后一个时间步的输出
        out = out[:, -1, :]
        # 批量归一化
        out = self.bn(out)
        # 全连接层
        out = self.fc(out)
        return out

# 用IC作为损失函数
def pearson_correlation(x, y):
    cov = torch.mean((x - torch.mean(x)) * (y - torch.mean(y)))
    std_x = torch.std(x)
    std_y = torch.std(y)
    return cov / (std_x * std_y)

class PearsonLoss(nn.Module):
    def __init__(self):
        super(PearsonLoss, self).__init__()

    def forward(self, x, y):
        return -pearson_correlation(x, y)

定义模型时，MSE和IC两种损失函数均有尝试

In [38]:
# 定义模型、损失函数和优化器
input_size = 10  # 特征数量
hidden_size = 64  # 隐藏层大小
num_layers = 2  # GRU层数
model = GRUNet(input_size, hidden_size, num_layers).to(device)
criterion = PearsonLoss()
#criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 模型训练参数
num_epochs = 5 #试过迭代10轮，但并不能有效提高验证集上的ic
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [39]:
start_time = datetime.now()
print("开始时间为:", start_time)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0  # 初始化整个 epoch 的损失
    total_batches = 0  # 初始化总批次数
    for features, target in train_loader:
        features, target = features.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs.squeeze(), target)
        #loss.requires_grad_(True)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()  # 累加当前批次的损失值
        total_batches += 1  # 更新总批次数

    epoch_loss = total_loss / total_batches
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-24 22:40:56.773534
Epoch [1/5], Loss: -0.0990
Epoch [2/5], Loss: -0.1234
Epoch [3/5], Loss: -0.1357
Epoch [4/5], Loss: -0.1435
Epoch [5/5], Loss: -0.1496
结束时间为: 2024-04-24 22:48:53.830518
处理时间为： 0:07:57.056984


In [40]:
start_time = datetime.now()
print("开始时间为:", start_time)

# 评估模型
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
model.eval()
predictions = []

with torch.no_grad():
    total_loss = 0
    for features, target in valid_loader:
        features, target = features.to(device), target.to(device)
        outputs = model(features)
        predictions.append(outputs)
        total_loss += criterion(outputs.squeeze(), target).item()
        
    print(f'Average Valid Loss: {total_loss / len(valid_loader):.4f}')

predictions = torch.cat(predictions, dim=0)
predictions_series = pd.Series(predictions.reshape(-1).cpu().numpy())
GRU_valid_df = GRU_valid_df_ori.copy()
GRU_valid_df['y_pred'] = predictions_series
GRU_rankic = model_function.get_rankic(GRU_valid_df)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-24 22:48:53.872520
Average Valid Loss: -0.0809
rankic均值为： 0.07884547028294218
结束时间为: 2024-04-24 22:49:07.132527
处理时间为： 0:00:13.260007


In [43]:
GRU_rankic

,date,RankIC
0,1541,0.013360
1,1542,0.051285
2,1543,0.103983
3,1544,0.140908
4,1545,0.166584
...,...,...
156,1697,0.066038
157,1698,0.049002
158,1699,0.004933
159,1700,-0.007969


In [44]:
GRU_rankic['RankIC'].mean() / GRU_rankic['RankIC'].std()

1.1043523536805642

In [45]:
class TestDataset(Dataset):
    def __init__(self, features):
        self.features = torch.tensor(features, dtype=torch.float)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx]

In [46]:
# 准备测试集数据
test_dataset = TestDataset(test_features)

# 创建测试集数据加载器
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [47]:
start_time = datetime.now()
print("开始时间为:", start_time)

# 进行预测
predictions = []
with torch.no_grad():
    for features in test_loader:
        features = features.to(device)
        outputs = model(features)
        predictions.append(outputs)

# 将所有预测结果连接成一个张量
predictions = torch.cat(predictions, dim=0)
predictions_series = pd.Series(predictions.reshape(-1).cpu().numpy())
GRU_test_result = GRU_test_result_ori.copy()
GRU_test_result['y_pred'] = predictions_series

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-24 22:51:42.014082
结束时间为: 2024-04-24 22:52:54.006391
处理时间为： 0:01:11.992309


In [48]:
GRU_test_result

,date,code,y_pred
0,1711,s_0,-0.220892
1,1712,s_0,-0.989944
2,1713,s_0,-1.382774
3,1714,s_0,-1.603202
4,1715,s_0,-0.425170
...,...,...,...
3828250,2598,s_999,0.129701
3828251,2599,s_999,0.048975
3828252,2600,s_999,0.105453
3828253,2601,s_999,0.119549


In [49]:
with open('./data/GRU_百分比_RZ_KNN_截面zscore的y.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('./data/GRU_百分比_RZ_KNN_截面zscore的y_test_result.pkl', 'wb') as f:
    pickle.dump(GRU_test_result, f)